In [ ]:
import JSON
using Plots
using HDF5
using LinearAlgebra
include("tools.jl")

In [ ]:
data = h5read("data/panta.h5","/")
mg = vec(data["gen_inertia"])
dg = vec(data["gen_prim_ctrl"])
idgen = Int64.(vec(data["gen"][:,1]))
dem = vec(data["bus"][:,3]) / 100
gen = vec(data["gen"][:,2]) / 100
coord = alberts_projection(data["bus_coord"]./(180/pi), 13.37616/180*pi, 46.94653/180*pi, 10/180*pi, 50/180*pi)
dl = vec(data["load_freq_coef"])
idb = Int64.(data["branch"][:,1:2])
bline = 1 ./ data["branch"][:,4]
th_ref = vec(data["bus"][:,9]/180*pi)

In [ ]:
data = h5read("eigen_20.h5","/")
c = data["coord"]
V = data["V"]
D = data["D"]
Ntrunc = 50
mu = zeros(Ntrunc)
pie = zeros(Ntrunc) #pie to avoid overloading pi ;-)
theta = zeros(Ntrunc)

for a in 1:Ntrunc
    println(a)
    for g in 1:length(mg)
        #println(g)
        #id = argmin((c[:,1] .- coord[idgen[g],1]).^2 + (c[:,2] .- coord[idgen[g],2]).^2)
        #mu[a] += mg[g] * V[id,a]
        #pie[a] += mg[g] * V[id,a]
        d = 10*sqrt.((c[:,1] .- coord[idgen[g],1]).^2 + (c[:,2] .- coord[idgen[g],2]).^2)
        f = exp.(-d) ./ sum(exp.(-d)) 
        #println(sum(f))
        #println(maximum(f))
        #println(V[id,a])        
        mu[a] += mg[g] * dot(f, V[:,a])
        pie[a] += mg[g] *  dot(f, V[:,a])
    end
    
    for l in 1:length(dl)
        #id = argmin((c[:,1] .- coord[l,1]).^2 + (c[:,2] .- coord[l,2]).^2)
        #pie[a] += dem[l] * V[id,a]
        #theta[a] += th_ref[l] * V[id,a]
        d = 10*sqrt.((c[:,1] .- coord[l,1]).^2 + (c[:,2] .- coord[l,2]).^2)
        f = exp.(-d) ./ sum(exp.(-d))
        pie[a] += dem[l] * dot(f,V[:,a])
        theta[a] += th_ref[l] * dot(f,V[:,a])
    end
end


In [ ]:
mu

In [ ]:
#mbis = sum(mg) * ones(size(c,1)) / size(c,1) 
mbis = zeros(size(c,1))
pbis = zeros(size(c,1))
thetabis = zeros(size(c,1))
mmin = Inf
a = 1
mthres = 0.1
#while(mmin > mthres &&  a < 100) 
for a = 1:50 
    mbis += mu[a] * V[:,a]
    pbis += pie[a] * V[:,a]
    thetabis += theta[a] * V[:,a]
    mmin = minimum(mbis)
    #a += 1
end
a

In [ ]:
Nx = data["Nx"]
Ny = data["Ny"]
isgrid = Bool.(data["isgrid"])
id_x = vec(data["id_x"])
id_y = vec(data["id_y"])
id_xy = Vector{CartesianIndex{2}}()
for i=1:length(id_x)
    push!(id_xy, CartesianIndex{2}((id_y[i],id_x[i])))
end
#id_xy = vec(data["id_xy"])
println(size(V[:,25]))
mlat = zeros(Ny, Nx)
plat = zeros(Ny, Nx)
thlat = zeros(Ny, Nx)
#mode[id_xy] = V_ref[:,3]
mlat[id_xy] = mbis
plat[id_xy] = pbis
thlat[id_xy] = thetabis

println(minimum(mlat))
#print(sum(mg))
println(sum(mlat))
mlat[.!isgrid] .= NaN
plat[.!isgrid] .= NaN
thlat[.!isgrid] .= NaN



contour(1:Nx, 1:Ny, mlat, fill=true)


In [ ]:
contour(1:Nx, 1:Ny, plat, fill=true)

In [ ]:
println(minimum(th_ref))
println(maximum(th_ref))
contour(1:Nx, 1:Ny, thlat, fill=true)

In [ ]:
plot(D[1:100])